In [2]:
import pandas as pd
import plotly.express as px

# =========================================================
# 1. Load data
# =========================================================
df = pd.read_csv('/content/doc-20251209-wa0000_')

# =========================================================
# 2. Clean "Customer Type"
# =========================================================
df['Customer Type'] = (
    df['Customer Type']
    .astype(str)
    .str.strip()
    .replace({'': 'Not Defined', 'nan': 'Not Defined', 'NaN': 'Not Defined'})
)
df['Customer Type'] = df['Customer Type'].fillna('Not Defined')

# =========================================================
# 3. Parse dates
# =========================================================
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Subset only 2025 for time-series analysis
df_year = df[df['Date'].dt.year == 2025].copy()
df_year['Month'] = df_year['Date'].dt.to_period('M').dt.to_timestamp()

# =========================================================
# 4. Color maps (consistent across plots)
# =========================================================
# For stacked bar / pie
custom_colors = {
    "Retail Collector": "#b38f00",
    "Gallery": "#cca300",
    "Museum": "#ffcc00",
    "Not Defined": "#ffdb4d",
    "Wholesaler": "#ffe680",
    "Interior Designer": "#fff5cc",
}

# For line / area / bubbles / ranking
color_map = {
    "Gallery": "#732626",
    "Interior Designer": "#ff1a1a",
    "Museum": "#cc5200",
    "Not Defined": "#0047b3",
    "Retail Collector": "#ffff66",
    "Wholesaler": "#2db300",
}

# =========================================================
# 5. Base aggregations
# =========================================================

# 5.1 Customer counts (all data, not only 2025)
customer_counts = (
    df['Customer Type']
    .value_counts()
    .rename_axis('Customer Type')
    .reset_index(name='Count')
    .sort_values('Count', ascending=True)
)
customer_counts['All'] = 'All Customers'

# 5.2 Time series (2025 only)
time_series = (
    df_year
    .groupby(['Month', 'Customer Type'])
    .size()
    .reset_index(name='Count')
)

# 5.3 Totals per customer type in 2025
customer_totals_2025 = (
    df_year
    .groupby('Customer Type')
    .size()
    .reset_index(name='Total Orders 2025')
)

# =========================================================
# 6. Visual 1 – Stacked bar chart (all orders by customer type)
# =========================================================
stack_fig = px.bar(
    customer_counts,
    x='All',
    y='Count',
    color='Customer Type',
    text='Count',
    color_discrete_map=custom_colors,
    category_orders={'Customer Type': customer_counts['Customer Type'].tolist()},
    title="Customer Segments by Type – Total Orders (All Years)",
    labels={'Count': 'Number of Orders', 'All': ''},
    height=600,
)

stack_fig.update_traces(
    textposition="inside",
    insidetextanchor="middle",
    width=0.45,
    marker_line_color="black",
)

stack_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    showlegend=True,
    legend=dict(
        title="Customer Type",
        traceorder="normal",
    ),
    xaxis=dict(showticklabels=False),
    yaxis_title="Number of Orders",
    margin=dict(l=20, r=0, t=40, b=20),
)

stack_fig.show()

# =========================================================
# 7. Visual 2 – Line chart: monthly evolution 2025
# =========================================================
line_fig = px.line(
    time_series,
    x='Month',
    y='Count',
    color='Customer Type',
    markers=True,
    color_discrete_map=color_map,
    title="Monthly Sales Performance by Customer Type – 2025",
    labels={'Count': 'Number of Orders', 'Month': ''},
)

line_fig.update_xaxes(
    dtick="M1",
    tickformat="%b",
    tickangle=-45,
)

line_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    yaxis_title="Number of Orders",
    margin=dict(l=20, r=20, t=50, b=50),
)

line_fig.show()

# =========================================================
# 8. Extra Visual – Donut chart (share of total orders by segment)
# =========================================================
segment_fig = px.pie(
    customer_counts,
    names='Customer Type',
    values='Count',
    hole=0.4,  # donut
    color='Customer Type',
    color_discrete_map=custom_colors,
    title="Customer Segments – Share of Total Orders (All Years)"
)

segment_fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

segment_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    legend_title="Customer Type",
    margin=dict(l=20, r=20, t=50, b=20),
)

segment_fig.show()

# =========================================================
# 9. Extra Visual – Heatmap: Month × Customer Type (2025)
# =========================================================
heatmap_data = (
    time_series
    .pivot(index='Customer Type', columns='Month', values='Count')
    .fillna(0)
)

heatmap_fig = px.imshow(
    heatmap_data,
    labels=dict(x="Month (2025)", y="Customer Type", color="Number of Orders"),
    title="Order Intensity by Customer Type and Month – 2025",
    aspect="auto"
)

# Make month labels nicer
heatmap_fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(len(heatmap_data.columns))),
    ticktext=[m.strftime('%b') for m in heatmap_data.columns]
)

heatmap_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    margin=dict(l=60, r=20, t=60, b=60),
)

heatmap_fig.show()

# =========================================================
# 10. Extra Visual – Area chart: share per month (normalized)
# =========================================================
share_ts = time_series.copy()
share_ts['Month_Total'] = share_ts.groupby('Month')['Count'].transform('sum')
share_ts['Share'] = share_ts['Count'] / share_ts['Month_Total']

area_fig = px.area(
    share_ts,
    x='Month',
    y='Share',
    color='Customer Type',
    color_discrete_map=color_map,
    groupnorm='fraction',  # normalize to 100% per month
    title="Monthly Share of Orders by Customer Type – 2025",
    labels={'Share': 'Share of Orders', 'Month': ''}
)

area_fig.update_xaxes(
    dtick="M1",
    tickformat="%b",
    tickangle=-45
)

area_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    yaxis_tickformat='.0%',
    margin=dict(l=20, r=20, t=50, b=50),
)

area_fig.show()

# =========================================================
# 11. Extra Visual – Cumulative orders by segment (2025)
# =========================================================
cum_ts = time_series.sort_values(['Customer Type', 'Month']).copy()
cum_ts['Cumulative Orders'] = cum_ts.groupby('Customer Type')['Count'].cumsum()

cum_fig = px.line(
    cum_ts,
    x='Month',
    y='Cumulative Orders',
    color='Customer Type',
    color_discrete_map=color_map,
    markers=True,
    title="Cumulative Orders by Customer Type – 2025",
    labels={'Cumulative Orders': 'Cumulative Number of Orders', 'Month': ''}
)

cum_fig.update_xaxes(
    dtick="M1",
    tickformat="%b",
    tickangle=-45
)

cum_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    margin=dict(l=20, r=20, t=50, b=50),
)

cum_fig.show()

# =========================================================
# 12. Extra Visual – Bubble chart: Month × Segment intensity (2025)
# =========================================================
bubble_fig = px.scatter(
    time_series,
    x='Month',
    y='Customer Type',
    size='Count',
    color='Customer Type',
    color_discrete_map=color_map,
    title="Order Intensity by Customer Type and Month – 2025 (Bubble Chart)",
    labels={'Count': 'Number of Orders', 'Month': ''},
    hover_data=['Count']
)

bubble_fig.update_xaxes(
    dtick="M1",
    tickformat="%b",
    tickangle=-45
)

bubble_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    margin=dict(l=40, r=20, t=60, b=60),
)

bubble_fig.show()

# =========================================================
# 13. Extra Visual – Ranking bar: total orders by segment in 2025
# =========================================================
segment_rank_2025 = (
    df_year
    .groupby('Customer Type')
    .size()
    .reset_index(name='Total Orders 2025')
    .sort_values('Total Orders 2025', ascending=True)
)

rank_fig = px.bar(
    segment_rank_2025,
    x='Total Orders 2025',
    y='Customer Type',
    orientation='h',
    color='Customer Type',
    color_discrete_map=color_map,
    title="Total Orders by Customer Type – 2025",
    text='Total Orders 2025',
    labels={'Total Orders 2025': 'Number of Orders', 'Customer Type': 'Segment'}
)

rank_fig.update_traces(
    textposition='outside'
)

rank_fig.update_layout(
    plot_bgcolor="#d9d9d9",
    paper_bgcolor="#ffffff",
    font=dict(color="#000000"),
    showlegend=False,
    margin=dict(l=80, r=20, t=50, b=20),
)

rank_fig.show()

# =========================================================
# 14. Extra Visual – Per-customer behavior (if Customer ID exists)
# =========================================================
candidate_cols = ['Customer ID', 'Customer_Id', 'Customer Name', 'Customer']
customer_id_col = next((c for c in candidate_cols if c in df_year.columns), None)

if customer_id_col:
    print(f"Using '{customer_id_col}' as customer identifier.")

    cust_seg = (
        df_year
        .groupby(['Customer Type', customer_id_col])
        .size()
        .reset_index(name='Orders per Customer')
    )

    seg_customer_stats = (
        cust_seg
        .groupby('Customer Type')['Orders per Customer']
        .agg(['mean', 'max', 'count'])
        .reset_index()
        .rename(columns={
            'mean': 'Avg Orders per Customer',
            'max': 'Max Orders (Single Customer)',
            'count': 'Number of Customers'
        })
    )

    cust_bar_fig = px.bar(
        seg_customer_stats,
        x='Customer Type',
        y='Avg Orders per Customer',
        color='Customer Type',
        color_discrete_map=color_map,
        title="Average Orders per Customer by Segment – 2025",
        text='Avg Orders per Customer',
        labels={'Customer Type': 'Segment'}
    )

    cust_bar_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

    cust_bar_fig.update_layout(
        plot_bgcolor="#d9d9d9",
        paper_bgcolor="#ffffff",
        font=dict(color="#000000"),
        showlegend=False,
        margin=dict(l=40, r=20, t=60, b=40),
    )

    cust_bar_fig.show()

else:
    print("No customer identifier column found (e.g. 'Customer ID' or 'Customer Name'). Skipping per-customer analysis.")

Using 'Customer Name' as customer identifier.


In [3]:
# =============================================================
# 1. IMPORTS & CONFIG
# =============================================================
import pandas as pd
import numpy as np
import plotly.express as px
from itertools import combinations
from typing import Optional

# ---- CONFIG: change these as needed ----
CSV_PATH = "/content/doc-20251209-wa0000_"
ANALYSIS_YEAR = 2025
SHOW_FIGURES = True  # set to False if you only want tables / prints, no charts


# =============================================================
# 2. MAIN ANALYSIS FUNCTION
# =============================================================
def main(
    csv_path: str = CSV_PATH,
    analysis_year: int = ANALYSIS_YEAR,
    show_figures: bool = SHOW_FIGURES,
    product_col_name: Optional[str] = None
):

    # ---------------------------------------------------------
    # 2. LOAD DATA
    # ---------------------------------------------------------
    df = pd.read_csv(csv_path)

    # ---------------------------------------------------------
    # 3. BASIC CLEANING
    # ---------------------------------------------------------

    # --- Clean Customer Type (flat, no nesting) ---
    df["Customer Type"] = df["Customer Type"].astype(str)
    df["Customer Type"] = df["Customer Type"].str.strip()
    df["Customer Type"] = df["Customer Type"].replace(
        {"": "Not Defined", "nan": "Not Defined", "NaN": "Not Defined"}
    )
    df["Customer Type"] = df["Customer Type"].fillna("Not Defined")

    # --- Parse Date ---
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # --- Year and Month ---
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.to_period("M").dt.to_timestamp()

    # Focus on selected year only
    df_year = df[df["Year"] == analysis_year].copy()
    if df_year.empty:
        raise ValueError(f"No records found for year {analysis_year}.")

    # ---------------------------------------------------------
    # 4. DETECT KEY COLUMNS
    # ---------------------------------------------------------

    # Common ID candidate names
    id_candidates = ["Customer ID", "Customer_Id", "Customer", "Customer Name"]

    # -------- PRODUCT COLUMN --------
    if product_col_name is not None:
        # User explicitly tells which column is the product
        if product_col_name in df_year.columns:
            product_col = product_col_name
            print(f"✅ Using user-specified product column: '{product_col}'")
        else:
            raise ValueError(
                f"product_col_name='{product_col_name}' not found in columns.\n"
                f"Available columns: {list(df_year.columns)}"
            )
    else:
        # Try automatically
        product_candidates = ["Product", "Product Name", "Item", "SKU", "Product_Title"]
        product_col = None

        # Exact match
        for c in product_candidates:
            if c in df_year.columns:
                product_col = c
                break

        # Case-insensitive match
        if product_col is None:
            cols_lower = {c.lower(): c for c in df_year.columns}
            for c in product_candidates:
                if c.lower() in cols_lower:
                    product_col = cols_lower[c.lower()]
                    break

        # Fuzzy by keywords in column name
        if product_col is None:
            keywords = [
                "product",
                "item",
                "sku",
                "service",
                "class",
                "membership",
                "pass",
            ]
            for c in df_year.columns:
                cname = c.lower()
                if any(k in cname for k in keywords):
                    product_col = c
                    print(
                        f"⚠️ Inferred '{product_col}' as product column based on name keywords."
                    )
                    break

        # Final fallback: first object/text column that is not Date/Customer Type/ID
        if product_col is None:
            ignore_cols = ["Customer Type", "Date", "Year", "Month"] + id_candidates
            obj_cols = [
                c
                for c in df_year.columns
                if (df_year[c].dtype == "object") and (c not in ignore_cols)
            ]
            if obj_cols:
                product_col = obj_cols[0]
                print(
                    f"⚠️ Using '{product_col}' as product column (fallback to first text column)."
                )
            else:
                raise ValueError(
                    "Could not infer a product column automatically.\n"
                    f"Available columns: {list(df_year.columns)}\n"
                    "You can fix this by passing product_col_name='YourColumnName' to main()."
                )

    print(f"📦 Product column detected: {product_col}")

    # -------- CUSTOMER ID COLUMN --------
    customer_id = None
    for c in id_candidates:
        if c in df_year.columns:
            customer_id = c
            break

    if customer_id is None:
        # Fallback synthetic ID
        df_year["CustomerID"] = df_year.groupby("Customer Type").cumcount()
        customer_id = "CustomerID"

    print(f"👤 Customer ID column used: {customer_id}")

    # -------- PRICE / REVENUE COLUMN --------
    price_candidates = ["Price", "Unit Price", "Amount", "Total", "Revenue"]
    price_col = None
    for c in price_candidates:
        if c in df_year.columns:
            price_col = c
            break

    if price_col is not None:
        df_year["Revenue"] = df_year[price_col]
    else:
        df_year["Revenue"] = 0.0  # if no price, revenue-based charts will just show 0

    # ---------------------------------------------------------
    # 5. BASIC AGGREGATIONS
    # ---------------------------------------------------------

    # Segment counts (year) – CLEAN VERSION
    segment_counts = (
        df_year.groupby("Customer Type")
        .size()
        .reset_index(name="Count")
    )

    # sanity check (optional)
    # print("segment_counts columns:", segment_counts.columns)

    # Monthly orders by segment
    time_series = (
        df_year.groupby(["Month", "Customer Type"])
        .size()
        .reset_index(name="Orders")
    )

    # Monthly total (not used in charts, but useful if needed)
    monthly_total = (
        df_year.groupby("Month")
        .size()
        .reset_index(name="Orders")
    )

    # ---------------------------------------------------------
    # 6. VISUALS – CUSTOMER SEGMENTS
    # ---------------------------------------------------------
    if show_figures:
        # Bar chart: total orders by segment
        fig_seg_bar = px.bar(
            segment_counts,
            x="Customer Type",
            y="Count",
            title=f"Total Orders by Customer Segment ({analysis_year})",
            text="Count",
        )
        fig_seg_bar.show()

        # Pie chart: share of segments
        fig_seg_pie = px.pie(
            segment_counts,
            names="Customer Type",
            values="Count",
            title=f"Share of Orders by Customer Segment ({analysis_year})",
            hole=0.4,
        )
        fig_seg_pie.show()

        # Line chart: monthly orders by segment
        fig_ts = px.line(
            time_series,
            x="Month",
            y="Orders",
            color="Customer Type",
            markers=True,
            title=f"Monthly Orders by Customer Segment – {analysis_year}",
        )
        fig_ts.update_xaxes(dtick="M1", tickformat="%b", tickangle=-45)
        fig_ts.show()

    # ---------------------------------------------------------
    # 7. PRODUCT × SEGMENT ANALYSIS
    # ---------------------------------------------------------

    # Orders per product & segment
    prod_seg = (
        df_year.groupby(["Customer Type", product_col])
        .size()
        .reset_index(name="Count")
    )

    # Heatmap: which segments buy which products
    heatmap_data = prod_seg.pivot(
        index="Customer Type", columns=product_col, values="Count"
    ).fillna(0)

    if show_figures:
        fig_heat = px.imshow(
            heatmap_data,
            labels=dict(x="Product", y="Customer Type", color="Orders"),
            title="Heatmap – Products Purchased by Customer Segment (Orders)",
        )
        fig_heat.show()

        # Treemap: segment → product
        fig_tree = px.treemap(
            prod_seg,
            path=["Customer Type", product_col],
            values="Count",
            title="Treemap – Products by Customer Segment (Order Volume)",
        )
        fig_tree.show()

        # Top products per segment
        top_products = (
            prod_seg.sort_values("Count", ascending=False)
            .groupby("Customer Type")
            .head(7)
        )

        fig_top_prod = px.bar(
            top_products,
            x="Count",
            y=product_col,
            color="Customer Type",
            orientation="h",
            title="Top Products per Customer Segment (by Orders)",
            text="Count",
        )
        fig_top_prod.show()

    # ---------------------------------------------------------
    # 8. REVENUE HEATMAP (SEGMENT × PRODUCT)
    # ---------------------------------------------------------
    rev_matrix = (
        df_year.groupby(["Customer Type", product_col])["Revenue"]
        .sum()
        .reset_index()
    )

    rev_heat_data = rev_matrix.pivot(
        index="Customer Type", columns=product_col, values="Revenue"
    ).fillna(0)

    if show_figures:
        fig_rev_heat = px.imshow(
            rev_heat_data,
            labels=dict(x="Product", y="Customer Type", color="Revenue"),
            title=f"Revenue Heatmap – Segment vs Product ({analysis_year})",
        )
        fig_rev_heat.show()

    # ---------------------------------------------------------
    # 9. RFM ANALYSIS (RECENCY, FREQUENCY, MONETARY)
    # ---------------------------------------------------------
    snapshot_date = df_year["Date"].max() + pd.Timedelta(days=1)

    rfm = df_year.groupby(customer_id).agg(
        {
            "Date": lambda x: (snapshot_date - x.max()).days,  # Recency
            product_col: "count",  # Frequency
            "Revenue": "sum",  # Monetary
        }
    ).reset_index()

    # Rename for consistency
    rfm.columns = ["CustomerID", "Recency", "Frequency", "Monetary"]

    if show_figures:
        fig_rfm = px.scatter(
            rfm,
            x="Recency",
            y="Frequency",
            size="Monetary",
            color="Monetary",
            title="RFM Scatter – Customer Behavior (Recency, Frequency, Monetary)",
            hover_data=["Monetary"],
        )
        fig_rfm.show()

    # ---------------------------------------------------------
    # 10. PRODUCT AFFINITY – FREQUENTLY BOUGHT TOGETHER
    # ---------------------------------------------------------
    pairs = []
    grouped = df_year.groupby(customer_id)[product_col]

    for _, basket in grouped:
        items = basket.dropna().unique()
        if len(items) > 1:
            for combo in combinations(items, 2):
                pairs.append(tuple(sorted(combo)))

    if len(pairs) > 0:
        pair_df = pd.DataFrame(pairs, columns=["Product A", "Product B"])
        pair_counts = pair_df.value_counts().reset_index(name="Count")

        if show_figures:
            fig_affinity = px.scatter(
                pair_counts.head(50),
                x="Product A",
                y="Product B",
                size="Count",
                color="Count",
                title="Product Affinity – Frequently Bought Together (Top 50 Pairs)",
                hover_data=["Count"],
            )
            fig_affinity.show()
    else:
        print("No product combinations found for affinity analysis.")

    # ---------------------------------------------------------
    # 11. SIMPLE PREDICTIVE INSIGHTS (NO PROPHET)
    # ---------------------------------------------------------

    # Segment momentum (growth)
    ts_seg = (
        df_year.groupby(["Month", "Customer Type"])
        .size()
        .reset_index(name="Orders")
        .sort_values(["Customer Type", "Month"])
    )

    seg_growth_list = []
    for seg in ts_seg["Customer Type"].unique():
        sub = ts_seg[ts_seg["Customer Type"] == seg]
        if len(sub) >= 4:
            first_part = sub.head(3)["Orders"].mean()
            last_part = sub.tail(3)["Orders"].mean()
            growth = last_part - first_part
            seg_growth_list.append((seg, growth))
        else:
            seg_growth_list.append((seg, 0))

    seg_growth_df = pd.DataFrame(seg_growth_list, columns=["Customer Type", "Growth"])

    if show_figures:
        fig_seg_growth = px.bar(
            seg_growth_df,
            x="Customer Type",
            y="Growth",
            title="Predicted Segment Momentum (approx. growth in orders)",
            text="Growth",
        )
        fig_seg_growth.show()

    # Product momentum
    ts_prod = (
        df_year.groupby(["Month", product_col])
        .size()
        .reset_index(name="Orders")
        .sort_values([product_col, "Month"])
    )

    prod_growth_list = []
    for p in ts_prod[product_col].unique():
        sub = ts_prod[ts_prod[product_col] == p]
        if len(sub) >= 4:
            first_part = sub.head(3)["Orders"].mean()
            last_part = sub.tail(3)["Orders"].mean()
            growth = last_part - first_part
            prod_growth_list.append((p, growth))
        else:
            prod_growth_list.append((p, 0))

    prod_growth_df = pd.DataFrame(prod_growth_list, columns=["Product", "Growth"])

    if show_figures:
        fig_prod_growth = px.bar(
            prod_growth_df.sort_values("Growth", ascending=False).head(15),
            x="Growth",
            y="Product",
            orientation="h",
            title="Predicted Product Momentum (Top Growing Products)",
            text="Growth",
        )
        fig_prod_growth.show()

    # ---------------------------------------------------------
    # 12. SIMPLE RISK MODEL & RECOMMENDATIONS
    # ---------------------------------------------------------
    rfm_norm = rfm.copy()

    for col in ["Recency", "Frequency", "Monetary"]:
        col_min = rfm_norm[col].min()
        col_max = rfm_norm[col].max()
        if col_max == col_min:
            rfm_norm[col + "_norm"] = 0
        else:
            rfm_norm[col + "_norm"] = (rfm_norm[col] - col_min) / (
                col_max - col_min
            )

    # Simple churn score: higher Recency, lower Frequency/Monetary => higher risk
    rfm_norm["Churn_Score"] = (
        0.6 * rfm_norm["Recency_norm"]
        - 0.2 * rfm_norm["Frequency_norm"]
        - 0.2 * rfm_norm["Monetary_norm"]
    )

    rfm_norm["Churn Risk"] = pd.qcut(
        rfm_norm["Churn_Score"],
        q=4,
        labels=["Low", "Medium", "High", "Very High"],
    )

    if show_figures:
        fig_churn = px.scatter(
            rfm_norm,
            x="Recency",
            y="Frequency",
            size="Monetary",
            color="Churn Risk",
            title="Churn Risk – Customers by Recency, Frequency & Monetary",
            hover_data=["Monetary", "Churn_Score"],
        )
        fig_churn.show()

    # High-value customers at high churn risk
    high_value_churn = rfm_norm[
        (rfm_norm["Monetary"] > rfm_norm["Monetary"].median())
        & (rfm_norm["Churn Risk"].isin(["High", "Very High"]))
    ]

    print("High-value customers to retain (top 10):")
    print(high_value_churn[["CustomerID", "Monetary", "Churn Risk"]].head(10))

    if show_figures and not high_value_churn.empty:
        fig_retain = px.bar(
            high_value_churn.head(15),
            x="CustomerID",
            y="Monetary",
            title="High-Value Customers at High Churn Risk – Retention Priority",
            text="Monetary",
        )
        fig_retain.show()

    # Recommended segments to prioritize based on growth + revenue
    seg_revenue = (
        df_year.groupby("Customer Type")["Revenue"]
        .sum()
        .reset_index(name="Total Revenue")
    )

    seg_recommend = seg_revenue.merge(seg_growth_df, on="Customer Type", how="left")
    seg_recommend["Score"] = (
        seg_recommend["Total Revenue"] * 0.6 + seg_recommend["Growth"] * 0.4
    )
    seg_recommend = seg_recommend.sort_values("Score", ascending=False)

    print("\nRecommended segments to prioritize:")
    print(seg_recommend.head(5))

    if show_figures:
        fig_seg_rec = px.bar(
            seg_recommend,
            x="Customer Type",
            y="Score",
            title="Recommended Segments to Prioritize (Revenue + Growth)",
            text="Score",
        )
        fig_seg_rec.show()

    # Recommended products to promote based on growth + revenue
    prod_revenue = (
        df_year.groupby(product_col)["Revenue"]
        .sum()
        .reset_index(name="Total Revenue")
        .rename(columns={product_col: "Product"})
    )

    prod_recommend = prod_revenue.merge(prod_growth_df, on="Product", how="left")
    prod_recommend["Score"] = (
        prod_recommend["Total Revenue"] * 0.6 + prod_recommend["Growth"] * 0.4
    )
    prod_recommend = prod_recommend.sort_values("Score", ascending=False)

    print("\nTop products to promote:")
    print(prod_recommend.head(10))

    if show_figures:
        fig_prod_rec = px.bar(
            prod_recommend.head(15),
            x="Score",
            y="Product",
            orientation="h",
            title="Top Products to Promote (Revenue + Growth)",
            text="Score",
        )
        fig_prod_rec.show()

    # Return key dataframes if you want to inspect them later
    return {
        "df_year": df_year,
        "segment_counts": segment_counts,
        "rfm": rfm,
        "rfm_norm": rfm_norm,
        "seg_recommend": seg_recommend,
        "prod_recommend": prod_recommend,
    }


# =============================================================
# 3. RUN SCRIPT
# =============================================================
if __name__ == "__main__":
    # If needed, you can force the product column like:
    # results = main(product_col_name="Service Name")
    results = main()

⚠️ Inferred 'Item ID' as product column based on name keywords.
📦 Product column detected: Item ID
👤 Customer ID column used: Customer Name


High-value customers to retain (top 10):
Empty DataFrame
Columns: [CustomerID, Monetary, Churn Risk]
Index: []

Recommended segments to prioritize:
       Customer Type  Total Revenue    Growth     Score
1  Interior Designer            0.0  1.000000  0.400000
3        Not Defined            0.0 -0.666667 -0.266667
5         Wholesaler            0.0 -0.666667 -0.266667
0            Gallery            0.0 -3.333333 -1.333333
2             Museum            0.0 -4.666667 -1.866667



Top products to promote:
     Product  Total Revenue  Growth  Score
1033  I-1034            0.0       0    0.0
0     I-0001            0.0       0    0.0
1     I-0002            0.0       0    0.0
2     I-0003            0.0       0    0.0
3     I-0004            0.0       0    0.0
4     I-0005            0.0       0    0.0
5     I-0006            0.0       0    0.0
6     I-0007            0.0       0    0.0
7     I-0008            0.0       0    0.0
8     I-0009            0.0       0    0.0
